﻿
# 重采样数据
## Pandas 重采样
你已经学习了将数据分成不同的时段，例如按月划分。我们来看看划分方式。我们从一系列天数开始。

In [1]:
import numpy as np
import pandas as pd

In [2]:
#用来创建日期
dates = pd.date_range('10/10/2018', periods=11, freq='D')
# 用来创建伪数据
close_prices = np.arange(len(dates))
close_prices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [3]:
close = pd.Series(close_prices, dates)
close

2018-10-10     0
2018-10-11     1
2018-10-12     2
2018-10-13     3
2018-10-14     4
2018-10-15     5
2018-10-16     6
2018-10-17     7
2018-10-18     8
2018-10-19     9
2018-10-20    10
Freq: D, dtype: int64

假设我们想要将这些天划分成 3 天时段我们将使用[DataFrame.resample](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.resample.html) 函数。该函数的第一个参数是叫做 `rule` 的字符串，表示如何重采样数据。此字符串表示法采用的是偏移别名。[这是](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases)偏移别名列表。为了创建 3 天时段，我们将 `rule` 设为“3D”。

In [4]:
close.resample('3D')

DatetimeIndexResampler [freq=<3 * Days>, axis=0, closed=left, label=left, convention=start, base=0]

上述代码会返回一个 `DatetimeIndexResampler` 对象。它是一个类似于 `GroupBy` 对象的中间对象。与 groupby 一样，它将原始数据分成小组。所以我们需要对这些组应用运算。为了简单起见，我们从每组中获取第一个元素。

In [5]:
close.resample('3D').first()

2018-10-10    0
2018-10-13    3
2018-10-16    6
2018-10-19    9
dtype: int64

你可能注意到了，这与 `.iloc[::3]` 是一样的

In [6]:
close.iloc[::3]

2018-10-10    0
2018-10-13    3
2018-10-16    6
2018-10-19    9
Freq: 3D, dtype: int64

为何要使用 `resample` 函数，而不是 `.iloc[::3]` 或 `groupby` 函数？

在处理时间和/或日期类任务时，`resample` 函数很有用。实际上，如果索引不是[与时间相关的类](https://pandas.pydata.org/pandas-docs/version/0.21/timeseries.html#overview)，你无法使用此函数。

In [7]:
try:
    # Attempt resample on a series without a time index
    pd.Series(close_prices).resample('W')
except TypeError:
    print('It threw a TypeError.')
else:
    print('It worked.')

It threw a TypeError.


它能够处理的重采样任务包括重采样时段，例如星期。我们将 `close` 从按天划分重采样为按星期划分。我们将使用“W”偏移别名，表示 Weeks。

In [8]:
# 上述数据中，间隔为7天的数据，只有以下两个，分别对应周一和周六（星期offset是按照从星期一开始，也就是0）
close.resample('W').first()

2018-10-14    0
2018-10-21    5
Freq: W-SUN, dtype: int64

In [9]:
pd.DataFrame({
    'days': close,
    'weeks': close.resample('W').first()})

,days,weeks
2018-10-10,0.0,NaN
2018-10-11,1.0,NaN
2018-10-12,2.0,NaN
2018-10-13,3.0,NaN
2018-10-14,4.0,0.0
2018-10-15,5.0,NaN
2018-10-16,6.0,NaN
2018-10-17,7.0,NaN
2018-10-18,8.0,NaN
2018-10-19,9.0,NaN


星期偏移量认为一星期从周一开始。因为 2018-10-10 是周三，所以第一组仅查看前 5 项。还有一些偏移量可以处理更复杂的问题，例如滤除节假日。暂时我们将仅重采样天、星期、月、季度和年。数据频率取决于交易频率。如果你根据仅在一年结束时才会出来的报告做出交易策略，那么只需考虑按年或按月划分。
## OHLC
你已经知道 Pandas 如何重采样时序数据，我们可以将它应用到开盘价、最高价、最低价和收盘价 (OHLC)。Pandas 提供了 [`Resampler.ohlc`](https://pandas.pydata.org/pandas-docs/version/0.21.0/generated/pandas.core.resample.Resampler.ohlc.html#pandas.core.resample.Resampler.ohlc) 函数，它能够将任何重采样频率转换为 OHLC 数据。我们来看看每周 OHLC。

In [10]:
close.resample('W').ohlc()

,open,high,low,close
2018-10-14,0,4,0,4
2018-10-21,5,10,5,10


你能发现潜在问题吗？它只能重采样已经重采样过的数据。

我们从收盘数据中获得了 OHLC。如果我们想从已经重采样的数据中获取 OHLC 数据，我们应该根据开盘价数据重采样第一个价格，并根据最高价数据重采样最高价，等等。

要从 `close` 获取每周收盘价，我们可以使用 [`Resampler.last`](https://pandas.pydata.org/pandas-docs/version/0.21.0/generated/pandas.core.resample.Resampler.last.html#pandas.core.resample.Resampler.last) 函数。

In [11]:
close.resample('W').last()

2018-10-14     4
2018-10-21    10
Freq: W-SUN, dtype: int64

## 小测验
实现 `days_to_weeks` 函数，并将 OHLC 价格数据重采样为每周 OHLC 价格数据。你可以在[此处](https://pandas.pydata.org/pandas-docs/version/0.21.0/api.html#id44)找到计算最高价和最低价的更多重采样函数。

In [14]:
import quiz_tests


def days_to_weeks(open_prices, high_prices, low_prices, close_prices):
    """Converts daily OHLC prices to weekly OHLC prices.
    
    Parameters
    ----------
    open_prices : DataFrame
        Daily open prices for each ticker and date
    high_prices : DataFrame
        Daily high prices for each ticker and date
    low_prices : DataFrame
        Daily low prices for each ticker and date
    close_prices : DataFrame
        Daily close prices for each ticker and date

    Returns
    -------
    open_prices_weekly : DataFrame
        Weekly open prices for each ticker and date
    high_prices_weekly : DataFrame
        Weekly high prices for each ticker and date
    low_prices_weekly : DataFrame
        Weekly low prices for each ticker and date
    close_prices_weekly : DataFrame
        Weekly close prices for each ticker and date
    """
    
    open_prices_weekly = open_prices.resample('W').first()
    high_prices_weekly = high_prices.resample('W').max()
    low_prices_weekly = low_prices.resample('W').min()
    close_prices_weekly = close_prices.resample('W').last()
    
    return open_prices_weekly, high_prices_weekly, low_prices_weekly, close_prices_weekly


quiz_tests.test_days_to_weeks(days_to_weeks)

Tests Passed


## 小测验解答
如果你遇到问题，请在[此处](resample_data_solution.ipynb)查看解答。